In [1]:
import scipy
# 多种纹理过滤器，并从各自响应中提取统计特征，最后将这些特征组合成最终的特征向量
import cv2
import numpy as np
def extract_stats(image: np.ndarray) -> list:

    stats = []
    # 计算平均值
    mean = np.mean(image)
    stats.append(mean)
    # 计算方差
    variance = np.var(image)
    stats.append(variance)
    # 计算最大最小值
    max_value = np.max(image)
    min_value = np.min(image)
    stats.extend([max_value, min_value])
    # 计算峰峰值
    peak_to_peak = max_value - min_value
    stats.append(peak_to_peak)
    # 计算偏斜度和峰度
    skewness1 = skew(image.ravel())
    kurtosis1 = kurtosis(image.ravel(), fisher=False)
    stats.extend([skewness1, kurtosis1])
    return stats

# 加载图像并转为灰度
biopsy_image = cv2.imread("C:\\Users\\MYZ\\PycharmProjects\\VIT_Pytorch\\BreaKHis_Datasets\\Original\\Benign_Sample\\Benign_Sample (2).png", cv2.IMREAD_GRAYSCALE)
# Gabor 滤波器参数及应用
scales = [1, 2, 4]  # 可调整的尺度列表
orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 四个典型方向
ksize = 15  # Gabor 滤波器核大小，一般取奇数，如 15！！
sigma = 1.0  # 核的标准偏差，与尺度有关，可以设置为尺度的函数
lambda_ = 4.0  # 波长，与尺度有关，可以设置为尺度的函数
gamma = 0.5  # 控制椭圆的扁平程度
psi = 0.0  # 相位偏移，一般设为 0
# 3. 应用 Gabor 滤波器并提取特征
gabor_responses = []
for scale in scales:
    for orientation in orientations:
        gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, orientation, lambda_, gamma, psi, ktype=cv2.CV_32F)
        filtered_image = cv2.filter2D(biopsy_image, cv2.CV_32F, gabor_kernel)
        gabor_responses.append(filtered_image)

# 拉普拉斯算子应用
laplacian = cv2.Laplacian(biopsy_image, cv2.CV_64F, ksize=3)

# 高斯差分滤波器参数及应用
# 定义两个不同的高斯核标准差
sigma_small = 1.0
sigma_large = 2.0
# 计算高斯模糊图像
blurred_small = cv2.GaussianBlur(biopsy_image, (0, 0), sigma_small)
blurred_large = cv2.GaussianBlur(biopsy_image, (0, 0), sigma_large)
# 计算 DoG 响应
dog_response = blurred_large - blurred_small

# Gabor 响应统计特征提取
gabor_stats = []
from scipy.stats import skew, kurtosis
   
for response in gabor_responses:
    mean = np.mean(response)
    variance = np.var(response)
    max_value = np.max(response)
    min_value = np.min(response)
    peak_to_peak = max_value - min_value
    # 计算偏斜度和峰度
    skewness1 = skew(response.ravel())  # 使用 ravel() 将图像展平为一维数组
    kurtosis1 = kurtosis(response.ravel(), fisher=False)  # 使用 fisher=False 获取 excess kurtosis（超额峰度）
    # 将偏斜度和峰度添加到当前响应的统计特征列表中
    current_stats = [mean, variance, max_value, min_value, peak_to_peak, skewness1, kurtosis1]
    gabor_stats.append(current_stats)

# 类似地，提取拉普拉斯算子和高斯差分滤波器的统计特征
laplacian_stats = extract_stats(laplacian)
dog_stats = extract_stats(dog_response)

# 组合特征向量
# 假设每个过滤器的统计特征数量为 n_stats
n_stats = len(gabor_stats[0])

# 初始化特征向量
combined_features = np.zeros((len(gabor_stats) + 2, n_stats))

# 将 Gabor、拉普拉斯算子和高斯差分滤波器的统计特征依次添加到特征向量
combined_features[:len(gabor_stats)] = np.array(gabor_stats)
combined_features[len(gabor_stats):len(gabor_stats) + 1] = laplacian_stats
combined_features[len(gabor_stats) + 1:] = dog_stats
# combined_features 现在包含了所有过滤器的统计特征，每一列代表一个统计指标，每一行代表一个过滤器的响应
combined_features

array([[ 6.78935730e+02,  6.59211084e+03,  8.86835693e+02,
         4.21505493e+02,  4.65330200e+02, -9.31966725e-02,
         2.61260887e+00],
       [ 6.79013184e+02,  6.80072021e+03,  9.03494568e+02,
         3.79014404e+02,  5.24480164e+02, -1.07457286e-01,
         2.66041860e+00],
       [ 6.78937500e+02,  6.59558838e+03,  9.28371765e+02,
         3.69021118e+02,  5.59350647e+02, -1.67773850e-01,
         2.89084259e+00],
       [ 6.79012695e+02,  6.12064795e+03,  8.93722656e+02,
         3.88007629e+02,  5.05715027e+02, -1.22044072e-01,
         2.80810522e+00],
       [ 6.78935730e+02,  6.59211084e+03,  8.86835693e+02,
         4.21505493e+02,  4.65330200e+02, -9.31966725e-02,
         2.61260887e+00],
       [ 6.79013184e+02,  6.80072021e+03,  9.03494568e+02,
         3.79014404e+02,  5.24480164e+02, -1.07457286e-01,
         2.66041860e+00],
       [ 6.78937500e+02,  6.59558838e+03,  9.28371765e+02,
         3.69021118e+02,  5.59350647e+02, -1.67773850e-01,
         2.8908425

In [2]:
import pandas as pd

column_names = ["Mean", "Variance", "Max", "Min", "Peak-to-Peak", "Skewness", "Kurtosis"]
df_combined_features = pd.DataFrame(combined_features, columns=column_names)
df_combined_features

,Mean,Variance,Max,Min,Peak-to-Peak,Skewness,Kurtosis
0,678.935730,6592.110840,886.835693,421.505493,465.330200,-0.093197,2.612609
1,679.013184,6800.720215,903.494568,379.014404,524.480164,-0.107457,2.660419
2,678.937500,6595.588379,928.371765,369.021118,559.350647,-0.167774,2.890843
3,679.012695,6120.647949,893.722656,388.007629,505.715027,-0.122044,2.808105
4,678.935730,6592.110840,886.835693,421.505493,465.330200,-0.093197,2.612609
5,679.013184,6800.720215,903.494568,379.014404,524.480164,-0.107457,2.660419
6,678.937500,6595.588379,928.371765,369.021118,559.350647,-0.167774,2.890843
7,679.012695,6120.647949,893.722656,388.007629,505.715027,-0.122044,2.808105
8,678.935730,6592.110840,886.835693,421.505493,465.330200,-0.093197,2.612609
9,679.013184,6800.720215,903.494568,379.014404,524.480164,-0.107457,2.660419
